In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
!pip install --upgrade beautifulsoup4

In [ ]:
import sqlalchemy
!pip install pymysql
import pymysql

In [ ]:
url = "https://en.wikipedia.org/wiki/List_of_European_cities_by_population_within_city_limits"

response = requests.get(url)
response.status_code

soup = BeautifulSoup(response.content, "html.parser")

#links for the cities
prefix = "https://en.wikipedia.org/wiki/"
all_links = []
for i in range(90):
  try:
    city_link = soup.find_all("tbody")[0].find_all("td")[i].find("a")["title"]
    all_links.append(prefix+city_link.replace(" ", "_"))
  except:
    continue

#filtering out links for countries
all_links_cl = []
for i in enumerate(all_links):
  if i[0] % 2 == 0:
    all_links_cl.append(i[1])
  else:
    continue

#scrapping the data
city_name = []
populations = []
for i in all_links_cl:
  response1 = requests.get(i)
  soup1 = BeautifulSoup(response1.content, "html.parser")
  city_name.append(soup1.select(".mw-page-title-main")[0].get_text())
  try:
    populations.append(soup1.select_one(".infobox-header:-soup-contains('Population')").parent.find_next_sibling().td.get_text())
  except:
    populations.append("NaN")

#converting to df
cities_tbl = pd.DataFrame({
    "city_name":city_names, 
    "populations":populations})

#adding date and time
from datetime import datetime
cities_tbl["time_stamps"] = datetime.now()

#cleaning
#dealing with extra characters in "population" column
cities_tbl["populations"] = cities_tbl.populations.str.replace(r"(\[\d\])", "").str.replace(",", "")

#dealing with non numerical data in "population" column
for i in range(10):
  if len(cities_tbl.populations[i]) > 15:
    cities_tbl.populations[i] = 0
  else:
    continue

cities_tbl.populations.inplace = True

#converting the datatype
import numpy as np
cities_tbl.populations = cities_tbl.populations.replace('NaN', np.nan).astype("float")

#CONNECTION
schema="P3_Cities"
host="city-project3-db.cf7wpiecfnwf.us-east-1.rds.amazonaws.com"
user="admin"
password="11768996Tom"
port=3306
con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

#push
cities_tbl.to_sql("city_populations",
                         if_exists='append',
                         con=con,
                         index=False)